In [19]:
# Import necessary packages
import ee
import geemap

In [20]:
# Initiate interactive map
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap('HYBRID') # Add Google Map
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [21]:
# This field contains UNIX time in milliseconds.
timeField = 'system:time_start'

In [22]:
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

In [23]:
roi = ee.Geometry.Point([-104.73081, 39.711584])

In [24]:
# Function to cloud mask from the pixel_qa band of Landsat 8 SR data.
# (From the Code Editor Examples > Cloud Masking)


def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(
        0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000).select('B[0-9]*').copyProperties(image, ['system:time_start'])

In [25]:
# Use this function to add variables for NDVI, time and a constant
# to Landsat 8 imagery.


def addVariables(image):
    # Compute time in fractional years since the epoch.
    date = ee.Date(image.get(timeField))
    years = date.difference(ee.Date('1970-01-01'), 'year')
    # Return the image with the added bands.
    return image.addBands(image.normalizedDifference(['B5', 'B4']).rename('NDVI')).addBands(
        ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

    # Purpose of methods to 'image' above in order applied
    # 1. Add an NDVI band.
    # 2. Add a time band.
    # 3. Add a constant band.

In [26]:
# Remove clouds, add variables (NDVI) and filter to the area of interest.
filteredLandsat = l8sr.filterBounds(roi).map(maskL8sr).map(addVariables)

##### Spectral unmixing (stuck- revisit later)
* Stuck because I switched to SR from TOA

In [17]:
# #  Add function to do spectral unmixing rather than NDVI


# # Load a Landsat 8 image and select the bands we want to unmix.
# def unmixL8sr(L8image):
# #     bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# #     image = L8image.select(bands)
# #     Map.addLayer(image, {'bands': ['B5', 'B4', 'B3'], min: 0, max: 128}, 'image')

#     # Define spectral endmembers.
#     urban = [88, 42, 48, 38, 86, 115, 59]
#     veg = [50, 21, 20, 35, 50, 110, 23]
#     water = [51, 20, 14, 9, 7, 116, 4]

#     # Unmix the image.
#     fractions = L8image.unmix([urban, veg, water])
#     Map.addLayer(fractions, {}, 'unmixed')

In [18]:
# # Remove clouds and unmix
# unmixedLandsat = l8sr.filterBounds(roi).map(maskL8sr).map(unmixL8sr)

EEException: ValueNode is empty

In [27]:
# Reduce the collection with the linear fit reducer.
# Independent variable are followed by dependent variables.
linearFit = filteredLandsat.select(['t', 'NDVI']).reduce(ee.Reducer.linearFit())

# Display the results.
Map.addLayer(linearFit, {'min': 0, 'max': [-0.9, 8e-5, 1], 'bands': ['scale', 'offset', 'scale']}, 'fit');

In [28]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [29]:
ndviParams = {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}


Map.addLayer(filteredLandsat.select('NDVI'), ndviParams, "NDVI")
Map.addLayer(roi)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:
# # ui.Chart is not available in the js or python API's.  Here is how to get around it: 
# # https://gis.stackexchange.com/questions/291823/ui-charts-for-indices-time-series-in-python-api-of-google-earth-engine

# Plot a time series of NDVI at a single location.
l8Chart = ui.Chart.image.series(filteredLandsat.select('NDVI'), roi).setChartType('ScatterChart').setOptions({
    title: 'Landsat 8 NDVI time series at ROI',
    trendlines: {0: {
        color: 'CC0000'
    }},
    lineWidth: 1,
    pointSize: 3,
})
print(l8Chart)

# List of the independent variable names
independents = ee.List(['constant', 't'])

# Name of the dependent variable.
dependent = ee.String('NDVI')

# Compute a linear trend.  This will have two bands: 'residuals' and
# a 2x1 band called coefficients (columns are for dependent variables).
trend = filteredLandsat.select(independents.add(dependent)).reduce(
    ee.Reducer.linearRegression(independents.length(), 1))
# Map.addLayer(trend, {}, 'trend array image')

# Flatten the coefficients into a 2-band image
coefficients = trend.select('coefficients').arrayProject(
    [0]).arrayFlatten([independents])
# Compute a de-trended series. ## Yet to convert form js to python.  Is this a lambda?
# detrended = filteredLandsat.map(function(image) {
#   return image.select(dependent).subtract(
#           image.select(independents).multiply(coefficients).reduce('sum'))
#           .rename(dependent)
#           .copyProperties(image, [timeField]);
# });

# Plot the detrended results.
detrendedChart = ui.Chart.image.series(detrended, roi, null, 30).setOptions({
    title: 'Detrended Landsat time series at ROI',
    lineWidth: 1,
    pointSize: 3,
})
print(detrendedChart)

#### How do I interpret spectral unmixing?

In [39]:
# Load a Landsat 5 image and select the bands we want to unmix.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
image = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA").filterBounds(roi).filterDate('2012-07-01', '2012-07-10').select(bands)
image = image.select('first')
Map.setCenter(-104.73081, 39.711584, 10); # Denver, CO
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], min: 0, max: 128}, 'image')

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])
Map.addLayer(fractions, {}, 'unmixed')


AttributeError: 'ImageCollection' object has no attribute 'unmix'

In [31]:
# Visualize 
Map

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…